In [1]:
!pip freeze | grep scikit-learn

scikit-learn==1.4.2
scikit-learn-intelex==2021.20220215.212715


In [2]:
!which python 
!python --version

/home/codespace/anaconda3/bin/python
Python 3.9.12


In [3]:
import pickle
import pandas as pd

In [11]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

/home/codespace/anaconda3/lib/python3.9/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator DictVectorizer from version 1.5.0 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/codespace/anaconda3/lib/python3.9/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LinearRegression from version 1.5.0 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [12]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [6]:
df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet')

In [13]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

# Q1. What's the standard deviation of the predicted duration for this dataset?

In [14]:
print(f"Standard deviation of predictions: {round(y_pred.std(axis=0), 2)}")

Standard deviation of predictions: 6.25


In [15]:
year = 2023
month = 3

df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [16]:
df_result = pd.DataFrame(data={"ride_id": df['ride_id'], "predictions": y_pred})
output_file = "predictions.parquet"
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

# Q2. What's the size of the output file

In [17]:
!ls -sh $output_file

66M predictions.parquet


# Q3. Creating the scoring script

```bash
jupyter nbconvert --to script homework.ipynb
```

# Q4. What's the first hash for the Scikit-Learn dependency?

In [18]:
!pipenv install scikit-learn==1.5.0 pyarrow pandas --python=3.10

Creating a virtualenv for this project...
Pipfile: /workspaces/MlOps_ZoomCamp_2024/week_4/Pipfile
Using /usr/local/python/3.10.13/bin/python3 (3.10.13) to create virtualenv...
⠴ Creating virtual environment...created virtual environment CPython3.10.13.final.0-64 in 1469ms
  creator CPython3Posix(dest=/home/codespace/.local/share/virtualenvs/week_4-l-ZtXWPa, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, wheel=bundle, via=copy, app_data_dir=/home/codespace/.local/share/virtualenv)
    added seed packages: pip==24.0, setuptools==69.1.0, wheel==0.42.0
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator

✔ Successfully created virtual environment!
⠴ Creating virtual environment...
Virtualenv location: /home/codespace/.local/share/virtualenvs/week_4-l-ZtXWPa
Creating a Pipfile for this project...
Installing scikit-learn==1.5.0...
Resolving scikit-learn==1.5.0...
Add

In [19]:
!cat Pipfile.lock

{
    "_meta": {
        "hash": {
            "sha256": "ec3d28266ce7ccf40369cbec2de7559c30ae47e84253039247a89b4f64de03b1"
        },
        "pipfile-spec": 6,
        "requires": {
            "python_full_version": "3.10.13",
            "python_version": "3.10"
        },
        "sources": [
            {
                "name": "pypi",
                "url": "https://pypi.org/simple",
                "verify_ssl": true
            }
        ]
    },
    "default": {
        "joblib": {
            "hashes": [
                "sha256:06d478d5674cbc267e7496a410ee875abd68e4340feff4490bcb7afb88060ae6",
                "sha256:2382c5816b2636fbd20a09e0f4e9dad4736765fdfb7dca582943b9c1366b3f0e"
            ],
            "markers": "python_version >= '3.8'",
            "version": "==1.4.2"
        },
        "numpy": {
            "hashes": [
                "sha256:04494f6ec467ccb5369d1808570ae55f6ed9b5809d7f035059000a37b8d7e86f",
                "sha256:0a43f0974d501842866cc83471bdb0

```
sha256:057b991ac64b3e75c9c04b5f9395eaf19a6179244c089afdebaad98264bff37c
```

# Q5 Parametrize the script

In [22]:
!pipenv run python score.py yellow 2023 4

loading model
reading file: https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-04.parquet
applying model
saving predictions
Mean of predictions: 14.29


In [25]:
!docker build -t ride_predictions .

[+] Building 0.0s (0/1)                                          docker:default
[+] Building 0.1s (2/2)                                          docker:default
 => [internal] load .dockerignore                                          0.1s
 => => transferring context: 2B                                            0.0s
 => [internal] load build definition from Dockerfile                       0.1s
 => => transferring dockerfile: 277B                                       0.0s
[+] Building 0.3s (2/3)                                          docker:default
 => [internal] load .dockerignore                                          0.1s
 => => transferring context: 2B                                            0.0s
 => [internal] load build definition from Dockerfile                       0.1s
 => => transferring dockerfile: 277B                                       0.0s
 => [internal] load metadata for docker.io/agrigorev/zoomcamp-model:mlops  0.2s
[+] Building 0.4s (2/3)                 

[+] Building 1.4s (4/11)                                         docker:default
 => [internal] load .dockerignore                                          0.1s
 => => transferring context: 2B                                            0.0s
 => [internal] load build definition from Dockerfile                       0.1s
 => => transferring dockerfile: 277B                                       0.0s
 => [internal] load metadata for docker.io/agrigorev/zoomcamp-model:mlops  0.4s
 => [1/7] FROM docker.io/agrigorev/zoomcamp-model:mlops-2024-3.10.13-slim  0.9s
 => => resolve docker.io/agrigorev/zoomcamp-model:mlops-2024-3.10.13-slim  0.1s
 => => sha256:f54535b73a8c3ef91967d5588de57d4e251b22addcb 1.78kB / 1.78kB  0.0s
 => => sha256:13e5353db264ec74c0d6d4d007bd6385ec849b6328e 7.34kB / 7.34kB  0.0s
 => => sha256:fb784af4aedaf9b1c591e0469ed0d893412349d29 12.38MB / 12.38MB  0.4s
 => => sha256:8a1e25ce7c4f75e372e9884f8f7b1bedcfe4a7a7d 29.12MB / 29.12MB  0.7s
 => => sha256:1103112ebfc46e01c0f35f3586

[+] Building 2.0s (4/11)                                         docker:default
 => [internal] load .dockerignore                                          0.1s
 => => transferring context: 2B                                            0.0s
 => [internal] load build definition from Dockerfile                       0.1s
 => => transferring dockerfile: 277B                                       0.0s
 => [internal] load metadata for docker.io/agrigorev/zoomcamp-model:mlops  0.4s
 => [1/7] FROM docker.io/agrigorev/zoomcamp-model:mlops-2024-3.10.13-slim  1.5s
 => => resolve docker.io/agrigorev/zoomcamp-model:mlops-2024-3.10.13-slim  0.1s
 => => sha256:f54535b73a8c3ef91967d5588de57d4e251b22addcb 1.78kB / 1.78kB  0.0s
 => => sha256:13e5353db264ec74c0d6d4d007bd6385ec849b6328e 7.34kB / 7.34kB  0.0s
 => => sha256:fb784af4aedaf9b1c591e0469ed0d893412349d29 12.38MB / 12.38MB  0.4s
 => => sha256:8a1e25ce7c4f75e372e9884f8f7b1bedcfe4a7a7d 29.12MB / 29.12MB  0.7s
 => => sha256:1103112ebfc46e01c0f35f3586

[+] Building 2.7s (4/11)                                         docker:default
 => [internal] load .dockerignore                                          0.1s
 => => transferring context: 2B                                            0.0s
 => [internal] load build definition from Dockerfile                       0.1s
 => => transferring dockerfile: 277B                                       0.0s
 => [internal] load metadata for docker.io/agrigorev/zoomcamp-model:mlops  0.4s
 => [1/7] FROM docker.io/agrigorev/zoomcamp-model:mlops-2024-3.10.13-slim  2.1s
 => => resolve docker.io/agrigorev/zoomcamp-model:mlops-2024-3.10.13-slim  0.1s
 => => sha256:f54535b73a8c3ef91967d5588de57d4e251b22addcb 1.78kB / 1.78kB  0.0s
 => => sha256:13e5353db264ec74c0d6d4d007bd6385ec849b6328e 7.34kB / 7.34kB  0.0s
 => => sha256:fb784af4aedaf9b1c591e0469ed0d893412349d29 12.38MB / 12.38MB  0.4s
 => => sha256:8a1e25ce7c4f75e372e9884f8f7b1bedcfe4a7a7d 29.12MB / 29.12MB  0.7s
 => => sha256:1103112ebfc46e01c0f35f3586

[+] Building 3.2s (4/11)                                         docker:default
 => [internal] load .dockerignore                                          0.1s
 => => transferring context: 2B                                            0.0s
 => [internal] load build definition from Dockerfile                       0.1s
 => => transferring dockerfile: 277B                                       0.0s
 => [internal] load metadata for docker.io/agrigorev/zoomcamp-model:mlops  0.4s
 => [1/7] FROM docker.io/agrigorev/zoomcamp-model:mlops-2024-3.10.13-slim  2.6s
 => => resolve docker.io/agrigorev/zoomcamp-model:mlops-2024-3.10.13-slim  0.1s
 => => sha256:f54535b73a8c3ef91967d5588de57d4e251b22addcb 1.78kB / 1.78kB  0.0s
 => => sha256:13e5353db264ec74c0d6d4d007bd6385ec849b6328e 7.34kB / 7.34kB  0.0s
 => => sha256:fb784af4aedaf9b1c591e0469ed0d893412349d29 12.38MB / 12.38MB  0.4s
 => => sha256:8a1e25ce7c4f75e372e9884f8f7b1bedcfe4a7a7d 29.12MB / 29.12MB  0.7s
 => => sha256:1103112ebfc46e01c0f35f3586

...

[+] Building 35.3s (11/12)                                       docker:default
 => => sha256:1103112ebfc46e01c0f35f3586e5a39c6a9ffa32c1a 3.51MB / 3.51MB  0.3s
 => => sha256:043e7f3dd05ab4bea31533694054d95fdc672fa08906624 245B / 245B  0.5s
 => => sha256:538a89c93346547bc2d83fb25dcd5e8bf44487152f1 3.36MB / 3.36MB  0.8s
 => => sha256:db87daa82210a671646131a94c07638f5e49051dc57e7882b 92B / 92B  0.8s
 => => extracting sha256:8a1e25ce7c4f75e372e9884f8f7b1bedcfe4a7a7d452eb4b  1.4s
 => => sha256:d8ee6bfe02f5a439eafdbf0c8d151061c2a115d3c28 8.32kB / 8.32kB  1.0s
 => => extracting sha256:1103112ebfc46e01c0f35f3586e5a39c6a9ffa32c1a362d4  0.2s
 => => extracting sha256:fb784af4aedaf9b1c591e0469ed0d893412349d29b10ccf7  0.6s
 => => extracting sha256:043e7f3dd05ab4bea31533694054d95fdc672fa089066248  0.0s
 => => extracting sha256:538a89c93346547bc2d83fb25dcd5e8bf44487152f1b0fcd  0.4s
 => => extracting sha256:db87daa82210a671646131a94c07638f5e49051dc57e7882  0.0s
 => => extracting sha256:d8ee6bfe02f5a43

[+] Building 35.9s (11/12)                                       docker:default
 => => sha256:1103112ebfc46e01c0f35f3586e5a39c6a9ffa32c1a 3.51MB / 3.51MB  0.3s
 => => sha256:043e7f3dd05ab4bea31533694054d95fdc672fa08906624 245B / 245B  0.5s
 => => sha256:538a89c93346547bc2d83fb25dcd5e8bf44487152f1 3.36MB / 3.36MB  0.8s
 => => sha256:db87daa82210a671646131a94c07638f5e49051dc57e7882b 92B / 92B  0.8s
 => => extracting sha256:8a1e25ce7c4f75e372e9884f8f7b1bedcfe4a7a7d452eb4b  1.4s
 => => sha256:d8ee6bfe02f5a439eafdbf0c8d151061c2a115d3c28 8.32kB / 8.32kB  1.0s
 => => extracting sha256:1103112ebfc46e01c0f35f3586e5a39c6a9ffa32c1a362d4  0.2s
 => => extracting sha256:fb784af4aedaf9b1c591e0469ed0d893412349d29b10ccf7  0.6s
 => => extracting sha256:043e7f3dd05ab4bea31533694054d95fdc672fa089066248  0.0s
 => => extracting sha256:538a89c93346547bc2d83fb25dcd5e8bf44487152f1b0fcd  0.4s
 => => extracting sha256:db87daa82210a671646131a94c07638f5e49051dc57e7882  0.0s
 => => extracting sha256:d8ee6bfe02f5a43

[+] Building 36.5s (11/12)                                       docker:default
 => => sha256:1103112ebfc46e01c0f35f3586e5a39c6a9ffa32c1a 3.51MB / 3.51MB  0.3s
 => => sha256:043e7f3dd05ab4bea31533694054d95fdc672fa08906624 245B / 245B  0.5s
 => => sha256:538a89c93346547bc2d83fb25dcd5e8bf44487152f1 3.36MB / 3.36MB  0.8s
 => => sha256:db87daa82210a671646131a94c07638f5e49051dc57e7882b 92B / 92B  0.8s
 => => extracting sha256:8a1e25ce7c4f75e372e9884f8f7b1bedcfe4a7a7d452eb4b  1.4s
 => => sha256:d8ee6bfe02f5a439eafdbf0c8d151061c2a115d3c28 8.32kB / 8.32kB  1.0s
 => => extracting sha256:1103112ebfc46e01c0f35f3586e5a39c6a9ffa32c1a362d4  0.2s
 => => extracting sha256:fb784af4aedaf9b1c591e0469ed0d893412349d29b10ccf7  0.6s
 => => extracting sha256:043e7f3dd05ab4bea31533694054d95fdc672fa089066248  0.0s
 => => extracting sha256:538a89c93346547bc2d83fb25dcd5e8bf44487152f1b0fcd  0.4s
 => => extracting sha256:db87daa82210a671646131a94c07638f5e49051dc57e7882  0.0s
 => => extracting sha256:d8ee6bfe02f5a43

[+] Building 37.1s (11/12)                                       docker:default
 => => sha256:1103112ebfc46e01c0f35f3586e5a39c6a9ffa32c1a 3.51MB / 3.51MB  0.3s
 => => sha256:043e7f3dd05ab4bea31533694054d95fdc672fa08906624 245B / 245B  0.5s
 => => sha256:538a89c93346547bc2d83fb25dcd5e8bf44487152f1 3.36MB / 3.36MB  0.8s
 => => sha256:db87daa82210a671646131a94c07638f5e49051dc57e7882b 92B / 92B  0.8s
 => => extracting sha256:8a1e25ce7c4f75e372e9884f8f7b1bedcfe4a7a7d452eb4b  1.4s
 => => sha256:d8ee6bfe02f5a439eafdbf0c8d151061c2a115d3c28 8.32kB / 8.32kB  1.0s
 => => extracting sha256:1103112ebfc46e01c0f35f3586e5a39c6a9ffa32c1a362d4  0.2s
 => => extracting sha256:fb784af4aedaf9b1c591e0469ed0d893412349d29b10ccf7  0.6s
 => => extracting sha256:043e7f3dd05ab4bea31533694054d95fdc672fa089066248  0.0s
 => => extracting sha256:538a89c93346547bc2d83fb25dcd5e8bf44487152f1b0fcd  0.4s
 => => extracting sha256:db87daa82210a671646131a94c07638f5e49051dc57e7882  0.0s
 => => extracting sha256:d8ee6bfe02f5a43

[+] Building 37.6s (11/12)                                       docker:default
 => => sha256:1103112ebfc46e01c0f35f3586e5a39c6a9ffa32c1a 3.51MB / 3.51MB  0.3s
 => => sha256:043e7f3dd05ab4bea31533694054d95fdc672fa08906624 245B / 245B  0.5s
 => => sha256:538a89c93346547bc2d83fb25dcd5e8bf44487152f1 3.36MB / 3.36MB  0.8s
 => => sha256:db87daa82210a671646131a94c07638f5e49051dc57e7882b 92B / 92B  0.8s
 => => extracting sha256:8a1e25ce7c4f75e372e9884f8f7b1bedcfe4a7a7d452eb4b  1.4s
 => => sha256:d8ee6bfe02f5a439eafdbf0c8d151061c2a115d3c28 8.32kB / 8.32kB  1.0s
 => => extracting sha256:1103112ebfc46e01c0f35f3586e5a39c6a9ffa32c1a362d4  0.2s
 => => extracting sha256:fb784af4aedaf9b1c591e0469ed0d893412349d29b10ccf7  0.6s
 => => extracting sha256:043e7f3dd05ab4bea31533694054d95fdc672fa089066248  0.0s
 => => extracting sha256:538a89c93346547bc2d83fb25dcd5e8bf44487152f1b0fcd  0.4s
 => => extracting sha256:db87daa82210a671646131a94c07638f5e49051dc57e7882  0.0s
 => => extracting sha256:d8ee6bfe02f5a43

# Q6. What's the mean predicted duration for May 2023?

In [26]:
!docker run ride_predictions /bin/bash -c "python score.py yellow 2023 5"

loading model
reading file: https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-05.parquet
applying model
saving predictions
Mean of predictions: 0.19
